In [1]:
import numpy as np
import awkward as ak
import fastjet
import h5py

def cluster_jets(event, jetdef, pt_min=20.0):
    
    event = ak.Array(event)
    print(event)
    
    cluster = fastjet.ClusterSequence(event, jetdef)    
    jets_ak = cluster.inclusive_jets(pt_min)
    jets = ak.to_numpy(jets_ak)

    #... get jet feature

    jet_pt = np.sqrt(jets['px']**2 + jets['py']**2)
    jet_rap = np.arctanh(jets['pz'] / jets['E'])
    jet_phi = np.arctan2(jets['py'], jets['px'])
    jet_M2 = jets['E']**2 - jets['px']**2 - jets['py']**2 - jets['pz']**2
    jet_M = np.sqrt(jet_M2[jet_M2 > 0])

    jets = np.array(list(zip(jet_pt, jet_rap, jet_phi, jet_M)))
    idx = np.argsort(-jets[0])  # The negative sign is to sort in descending order
    return jets[idx]

In [47]:

jet_def = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
f = h5py.File('../../../../data/jetclass/qcd_top_jets/top_N30_100k.hdf5', 'r')

R, pt_jet_min = 0.4, 0.0
jetdef = fastjet.JetDefinition(fastjet.kt_algorithm, R)
jets=[]
for ev in list(f['4_momenta'][..., :4]):
    px, py, pz, E = ev[:, 0], ev[:, 1], ev[:, 2], ev[:, 3]
    event = ak.zip({"px": px, "py": py, "pz": pz, "E": E}, with_name="Momentum4D")
    found_jets = fastjet.ClusterSequence(event, jetdef).inclusive_jets(pt_jet_min)
    jets.append(found_jets)
    print(found_jets[0])
    print(found_jets[1])
    print(found_jets[2])
    break



{px: -181, py: -354, pz: 770, E: 866}
{px: -44.9, py: -103, pz: 95.8, E: 148}
{px: 1.21, py: -35.4, pz: 54.5, E: 65.4}


In [50]:
len(event)

30

In [46]:
for event in events:
    fastjet_event = []
    print(event)
    for p in event:
        fastjet_event.append( fastjet.PseudoJet(p["px"], p["py"], p["pz"], p["E"]) )
    cluster = fastjet.ClusterSequence(fastjet_event, jetdef)
    jets = cluster.inclusive_jets()
    break

{px: -139, py: 9.11, pz: -347, E: 374}


TypeError: 'Record' object is not iterable

In [77]:
jetdef = fastjet.JetDefinition(fastjet.kt_algorithm, 0.4)
jet = fastjet.ClusterSequence(events[2], jetdef).inclusive_jets(20.0)

In [31]:
jets[0][0].const()

AttributeError: no field named 'const'